In [1]:
import numpy as np
import pandas as pd
import random
import time
from datetime import datetime, timedelta
import copy

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

class ExamScheduler:
    def __init__(self, courses_file, students_file, teachers_file, student_courses_file):
        # Load data
        self.courses = pd.read_csv(courses_file)
        self.students = pd.read_csv(students_file)
        self.teachers = pd.read_csv(teachers_file)
        self.student_courses = pd.read_csv(student_courses_file)
        
        # Process data
        self.course_list = self.courses['Course Code'].unique().tolist()
        self.student_list = self.students['Names'].unique().tolist()
        self.teacher_list = self.teachers['Names'].unique().tolist()
        
        # Map students to their courses
        self.student_to_courses = {}
        for _, row in self.student_courses.iterrows():
            student = row['Student Name']
            course = row['Course Code']
            if student not in self.student_to_courses:
                self.student_to_courses[student] = []
            self.student_to_courses[student].append(course)
        
        # Available rooms
        self.rooms = [f'C{i}' for i in range(301, 311)]
        
        # Available days and times
        self.days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        self.time_slots = [9, 10, 11, 12, 13, 14, 15, 16]  # 9 AM to 4 PM (exams can end at 5 PM)
        
        # Population parameters
        self.population_size = 50
        self.generations = 100
        self.mutation_rate = 0.1
        self.crossover_rate = 0.8
        
        # Keep track of the best solution
        self.best_fitness = -float('inf')
        self.best_schedule = None
        
        # Course durations (default 1 hour for all courses for simplicity)
        self.course_durations = {course: 1 for course in self.course_list}
        
        # Identify course type (CS or MG)
        self.course_types = {}
        for course in self.course_list:
            if course.startswith('CS'):
                self.course_types[course] = 'CS'
            elif course.startswith('MG'):
                self.course_types[course] = 'MG'
            else:
                self.course_types[course] = 'Other'

    def create_individual(self):
        """Create a random exam schedule"""
        schedule = {}
        # Shuffle available teachers, times, and rooms for random assignment
        available_teachers = self.teacher_list.copy()
        random.shuffle(available_teachers)
        
        # For each course, assign a random day, time, room, and teacher
        for course in self.course_list:
            day = random.choice(self.days)
            time_slot = random.choice(self.time_slots)
            room = random.choice(self.rooms)
            teacher = random.choice(available_teachers)  # Will be further checked in fitness function
            
            schedule[course] = {
                'day': day,
                'time': time_slot,
                'room': room,
                'teacher': teacher,
                'duration': self.course_durations[course]
            }
        
        return schedule

    def create_initial_population(self):
        """Create the initial population of schedules"""
        return [self.create_individual() for _ in range(self.population_size)]

    def evaluate_fitness(self, schedule):
        """Evaluate the fitness of a schedule"""
        penalties = 0
        rewards = 0
        
        # Track teacher and room assignments
        teacher_assignments = {}  # {(day, time): teacher}
        room_assignments = {}     # {(day, time): room}
        student_exams = {}        # {(student, day, time): course}
        
        # Check hard constraints
        for course, details in schedule.items():
            day = details['day']
            time = details['time']
            room = details['room']
            teacher = details['teacher']
            duration = details['duration']
            
            # Check for teacher conflicts
            for t in range(time, time + duration):
                key = (day, t)
                if key in teacher_assignments and teacher_assignments[key] == teacher:
                    penalties += 100  # Teacher already assigned at this time
                teacher_assignments[key] = teacher
                
                # Check for consecutive invigilation
                next_slot_key = (day, t + 1)
                if next_slot_key in teacher_assignments and teacher_assignments[next_slot_key] == teacher:
                    penalties += 50  # Teacher has consecutive invigilation
            
            # Check for room conflicts
            for t in range(time, time + duration):
                key = (day, t)
                if key in room_assignments and room_assignments[key] == room:
                    penalties += 100  # Room already booked at this time
                room_assignments[key] = room
            
            # Check for student conflicts (no overlapping exams)
            students_in_course = self.student_courses[self.student_courses['Course Code'] == course]['Student Name'].tolist()
            for student in students_in_course:
                for t in range(time, time + duration):
                    key = (student, day, t)
                    if key in student_exams:
                        penalties += 200  # Student has multiple exams at the same time
                    student_exams[key] = course
        
        # Check soft constraints
        
        # 1. Common break on Friday from 1-2 PM
        friday_1pm_exams = [(c, d) for c, d in schedule.items() if d['day'] == 'Friday' and d['time'] == 13]
        if not friday_1pm_exams:
            rewards += 10  # No exams during Friday 1-2 PM
        
        # 2. Students should not have back-to-back exams
        back_to_back_count = 0
        for student in self.student_to_courses:
            student_courses = self.student_to_courses.get(student, [])
            for i, course1 in enumerate(student_courses):
                if course1 not in schedule:
                    continue
                day1 = schedule[course1]['day']
                time1 = schedule[course1]['time']
                end_time1 = time1 + schedule[course1]['duration']
                
                for course2 in student_courses[i+1:]:
                    if course2 not in schedule:
                        continue
                    day2 = schedule[course2]['day']
                    time2 = schedule[course2]['time']
                    
                    if day1 == day2 and (time2 == end_time1 or time1 == time2 + schedule[course2]['duration']):
                        back_to_back_count += 1
        
        if back_to_back_count > 0:
            penalties += back_to_back_count * 5
        else:
            rewards += 10
        
        # 3. MG courses before CS courses
        mg_cs_order_violations = 0
        for student in self.student_to_courses:
            mg_courses = [c for c in self.student_to_courses.get(student, []) if c.startswith('MG')]
            cs_courses = [c for c in self.student_to_courses.get(student, []) if c.startswith('CS')]
            
            for mg_course in mg_courses:
                if mg_course not in schedule:
                    continue
                mg_day_idx = self.days.index(schedule[mg_course]['day'])
                mg_time = schedule[mg_course]['time']
                
                for cs_course in cs_courses:
                    if cs_course not in schedule:
                        continue
                    cs_day_idx = self.days.index(schedule[cs_course]['day'])
                    cs_time = schedule[cs_course]['time']
                    
                    # Check if CS course is before MG course
                    if cs_day_idx < mg_day_idx or (cs_day_idx == mg_day_idx and cs_time < mg_time):
                        mg_cs_order_violations += 1
        
        if mg_cs_order_violations > 0:
            penalties += mg_cs_order_violations * 5
        else:
            rewards += 10
        
        # 4. Faculty meeting break (two-hour break during the week)
        # For simplicity, check if there's at least one two-hour slot where < 50% of faculty is scheduled
        faculty_meeting_possible = False
        for day in self.days:
            for start_time in range(9, 15):  # End time would be start_time + 2
                teachers_busy = set()
                for course, details in schedule.items():
                    if details['day'] == day and start_time <= details['time'] < start_time + 2:
                        teachers_busy.add(details['teacher'])
                
                if len(teachers_busy) < len(self.teacher_list) / 2:
                    faculty_meeting_possible = True
                    break
            
            if faculty_meeting_possible:
                break
        
        if faculty_meeting_possible:
            rewards += 10
        else:
            penalties += 20
        
        # Calculate final fitness score
        fitness = 1000 - penalties + rewards
        
        return fitness

    def selection(self, population, fitnesses):
        """Roulette wheel selection"""
        # Adjust fitnesses to be positive
        min_fitness = min(fitnesses)
        if min_fitness < 0:
            adjusted_fitnesses = [f - min_fitness + 1 for f in fitnesses]
        else:
            adjusted_fitnesses = [f + 1 for f in fitnesses]
        
        # Calculate selection probabilities
        total_fitness = sum(adjusted_fitnesses)
        probabilities = [f / total_fitness for f in adjusted_fitnesses]
        
        # Select two parents using roulette wheel
        selected_indices = np.random.choice(
            range(len(population)), 
            size=2, 
            replace=False, 
            p=probabilities
        )
        
        return [copy.deepcopy(population[i]) for i in selected_indices]

    def crossover(self, parent1, parent2):
        """Single point crossover"""
        if random.random() > self.crossover_rate:
            return parent1, parent2
        
        # Create child schedules
        child1 = {}
        child2 = {}
        
        # Get a random crossover point
        courses = list(parent1.keys())
        crossover_point = random.randint(1, len(courses) - 1)
        
        # Create children by crossing over parents
        for i, course in enumerate(courses):
            if i < crossover_point:
                child1[course] = parent1[course]
                child2[course] = parent2[course]
            else:
                child1[course] = parent2[course]
                child2[course] = parent1[course]
        
        return child1, child2

    def mutate(self, schedule):
        """Mutate a schedule by changing random attributes"""
        for course in schedule:
            if random.random() < self.mutation_rate:
                # Choose a random attribute to mutate
                attribute = random.choice(['day', 'time', 'room', 'teacher'])
                
                if attribute == 'day':
                    schedule[course]['day'] = random.choice(self.days)
                elif attribute == 'time':
                    schedule[course]['time'] = random.choice(self.time_slots)
                elif attribute == 'room':
                    schedule[course]['room'] = random.choice(self.rooms)
                elif attribute == 'teacher':
                    schedule[course]['teacher'] = random.choice(self.teacher_list)
        
        return schedule

    def evolve(self):
        """Run the genetic algorithm"""
        # Create initial population
        print("Creating initial population...")
        population = self.create_initial_population()
        
        # Evaluate initial population
        fitnesses = [self.evaluate_fitness(schedule) for schedule in population]
        best_idx = fitnesses.index(max(fitnesses))
        self.best_schedule = copy.deepcopy(population[best_idx])
        self.best_fitness = fitnesses[best_idx]
        
        print(f"Initial best fitness: {self.best_fitness}")
        
        # Evolution loop
        for generation in range(self.generations):
            start_time = time.time()
            
            new_population = []
            
            # Elitism: keep the best individual
            elite_idx = fitnesses.index(max(fitnesses))
            new_population.append(copy.deepcopy(population[elite_idx]))
            
            # Create new individuals through selection, crossover, and mutation
            while len(new_population) < self.population_size:
                # Selection
                parents = self.selection(population, fitnesses)
                
                # Crossover
                child1, child2 = self.crossover(parents[0], parents[1])
                
                # Mutation
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                
                # Add to new population
                new_population.append(child1)
                if len(new_population) < self.population_size:
                    new_population.append(child2)
            
            # Update population
            population = new_population
            
            # Evaluate new population
            fitnesses = [self.evaluate_fitness(schedule) for schedule in population]
            
            # Update best solution if found
            generation_best_idx = fitnesses.index(max(fitnesses))
            generation_best_fitness = fitnesses[generation_best_idx]
            
            if generation_best_fitness > self.best_fitness:
                self.best_fitness = generation_best_fitness
                self.best_schedule = copy.deepcopy(population[generation_best_idx])
            
            # Print progress
            elapsed = time.time() - start_time
            print(f"Generation {generation+1}/{self.generations}: Best Fitness = {self.best_fitness}, Time: {elapsed:.2f}s")
            
            # Print the top 3 schedules of this generation
            top_indices = sorted(range(len(fitnesses)), key=lambda i: fitnesses[i], reverse=True)[:3]
            print("Top 3 schedules in this generation:")
            for rank, idx in enumerate(top_indices):
                print(f"  Rank {rank+1}: Fitness = {fitnesses[idx]}")
            
            # Early termination if perfect solution found
            if self.best_fitness >= 1000:
                print("Perfect solution found! Terminating early.")
                break
        
        print(f"Best fitness achieved: {self.best_fitness}")
        return self.best_schedule

    def format_schedule(self, schedule):
        """Format the schedule for output"""
        formatted = []
        
        for course, details in schedule.items():
            formatted.append({
                'Course': course,
                'Day': details['day'],
                'Time': f"{details['time']}:00",
                'Room': details['room'],
                'Teacher': details['teacher'],
                'Duration': f"{details['duration']} hour(s)"
            })
        
        # Sort by day and time
        day_order = {day: i for i, day in enumerate(self.days)}
        formatted.sort(key=lambda x: (day_order[x['Day']], int(x['Time'].split(':')[0])))
        
        return pd.DataFrame(formatted)

    def check_constraints(self, schedule):
        """Check which constraints are satisfied by the schedule"""
        # Track assignments
        teacher_assignments = {}  # {(day, time): teacher}
        room_assignments = {}     # {(day, time): room}
        student_exams = {}        # {(student, day, time): course}
        
        # Hard constraints
        hard_constraints = {
            "Every course has an exam": True,
            "No student has overlapping exams": True,
            "Exams only on weekdays": True,
            "Exam timings between 9 AM and 5 PM": True,
            "Each exam has an invigilating teacher": True,
            "No teacher assigned to multiple exams simultaneously": True,
            "No consecutive invigilation duties": True
        }
        
        # Soft constraints
        soft_constraints = {
            "Friday break from 1-2 PM": True,
            "No back-to-back exams for students": True,
            "MG courses before CS courses": True,
            "Two-hour faculty meeting break": False  # Default to False, will check later
        }
        
        # Check if all courses are scheduled
        scheduled_courses = set(schedule.keys())
        if set(self.course_list) != scheduled_courses:
            hard_constraints["Every course has an exam"] = False
        
        # Check each exam's constraints
        for course, details in schedule.items():
            day = details['day']
            time = details['time']
            room = details['room']
            teacher = details['teacher']
            duration = details['duration']
            
            # Check if exams are on weekdays
            if day not in self.days:
                hard_constraints["Exams only on weekdays"] = False
            
            # Check if exam timings are valid
            if time not in self.time_slots or time + duration > 17:
                hard_constraints["Exam timings between 9 AM and 5 PM"] = False
            
            # Check teacher assignments
            for t in range(time, time + duration):
                key = (day, t)
                if key in teacher_assignments and teacher_assignments[key] != teacher:
                    hard_constraints["No teacher assigned to multiple exams simultaneously"] = False
                teacher_assignments[key] = teacher
                
                # Check for consecutive invigilation
                next_slot_key = (day, t + 1)
                if next_slot_key in teacher_assignments and teacher_assignments[next_slot_key] == teacher:
                    hard_constraints["No consecutive invigilation duties"] = False
            
            # Check room assignments
            for t in range(time, time + duration):
                key = (day, t)
                if key in room_assignments and room_assignments[key] != room:
                    pass  # We're not checking for room conflicts as it's implicitly handled
                room_assignments[key] = room
            
            # Check student exam conflicts
            students_in_course = self.student_courses[self.student_courses['Course Code'] == course]['Student Name'].tolist()
            for student in students_in_course:
                for t in range(time, time + duration):
                    key = (student, day, t)
                    if key in student_exams and student_exams[key] != course:
                        hard_constraints["No student has overlapping exams"] = False
                    student_exams[key] = course
        
        # Check soft constraints
        
        # 1. Friday break from 1-2 PM
        for course, details in schedule.items():
            if details['day'] == 'Friday' and details['time'] == 13:
                soft_constraints["Friday break from 1-2 PM"] = False
                break
        
        # 2. No back-to-back exams for students
        for student in self.student_to_courses:
            student_courses = self.student_to_courses.get(student, [])
            student_exams = []
            
            for course in student_courses:
                if course in schedule:
                    day = schedule[course]['day']
                    time = schedule[course]['time']
                    end_time = time + schedule[course]['duration']
                    student_exams.append((day, time, end_time))
            
            # Sort by day and time
            student_exams.sort()
            
            # Check for back-to-back exams
            for i in range(len(student_exams) - 1):
                day1, _, end_time1 = student_exams[i]
                day2, time2, _ = student_exams[i + 1]
                
                if day1 == day2 and time2 == end_time1:
                    soft_constraints["No back-to-back exams for students"] = False
                    break
        
        # 3. MG courses before CS courses
        for student in self.student_to_courses:
            mg_courses = [c for c in self.student_to_courses.get(student, []) if c.startswith('MG')]
            cs_courses = [c for c in self.student_to_courses.get(student, []) if c.startswith('CS')]
            
            for mg_course in mg_courses:
                if mg_course not in schedule:
                    continue
                mg_day_idx = self.days.index(schedule[mg_course]['day'])
                mg_time = schedule[mg_course]['time']
                
                for cs_course in cs_courses:
                    if cs_course not in schedule:
                        continue
                    cs_day_idx = self.days.index(schedule[cs_course]['day'])
                    cs_time = schedule[cs_course]['time']
                    
                    # Check if CS course is before MG course
                    if cs_day_idx < mg_day_idx or (cs_day_idx == mg_day_idx and cs_time < mg_time):
                        soft_constraints["MG courses before CS courses"] = False
                        break
        
        # 4. Check if there's a two-hour faculty meeting break
        for day in self.days:
            for start_time in range(9, 15):  # End time would be start_time + 2
                teachers_busy = set()
                for course, details in schedule.items():
                    if details['day'] == day and start_time <= details['time'] < start_time + 2:
                        teachers_busy.add(details['teacher'])
                
                if len(teachers_busy) < len(self.teacher_list) / 2:
                    soft_constraints["Two-hour faculty meeting break"] = True
                    break
        
        return hard_constraints, soft_constraints

    def run(self):
        """Run the algorithm and display results"""
        print("Starting the exam scheduler...")
        
        # Run the genetic algorithm
        best_schedule = self.evolve()
        
        # Format and display the best schedule
        formatted_schedule = self.format_schedule(best_schedule)
        print("\nBest Schedule:")
        print(formatted_schedule)
        
        # Check constraints
        hard_constraints, soft_constraints = self.check_constraints(best_schedule)
        
        print("\nConstraints Check:")
        print("Hard Constraints:")
        for constraint, satisfied in hard_constraints.items():
            status = "✓" if satisfied else "✗"
            print(f"  {status} {constraint}")
        
        print("\nSoft Constraints:")
        for constraint, satisfied in soft_constraints.items():
            status = "✓" if satisfied else "✗"
            print(f"  {status} {constraint}")
        
        return formatted_schedule, hard_constraints, soft_constraints

# Main function to run the scheduler
def main():
    # File paths
    courses_file = 'courses.csv'
    students_file = 'studentNames.csv'
    teachers_file = 'teachers.csv'
    student_courses_file = 'studentCourse.csv'
    
    # Initialize and run the scheduler
    scheduler = ExamScheduler(courses_file, students_file, teachers_file, student_courses_file)
    formatted_schedule, hard_constraints, soft_constraints = scheduler.run()
    
    # Save the schedule to a CSV file
    formatted_schedule.to_csv('exam_schedule.csv', index=False)
    print("\nSchedule saved to 'exam_schedule.csv'")
    
    return formatted_schedule, hard_constraints, soft_constraints

if __name__ == "__main__":
    main()

Starting the exam scheduler...
Creating initial population...
Initial best fitness: 450
Generation 1/100: Best Fitness = 450, Time: 0.45s
Top 3 schedules in this generation:
  Rank 1: Fitness = 450
  Rank 2: Fitness = 450
  Rank 3: Fitness = 440
Generation 2/100: Best Fitness = 535, Time: 0.43s
Top 3 schedules in this generation:
  Rank 1: Fitness = 535
  Rank 2: Fitness = 455
  Rank 3: Fitness = 450
Generation 3/100: Best Fitness = 535, Time: 0.45s
Top 3 schedules in this generation:
  Rank 1: Fitness = 535
  Rank 2: Fitness = 535
  Rank 3: Fitness = 535
Generation 4/100: Best Fitness = 565, Time: 0.73s
Top 3 schedules in this generation:
  Rank 1: Fitness = 565
  Rank 2: Fitness = 550
  Rank 3: Fitness = 535
Generation 5/100: Best Fitness = 575, Time: 1.39s
Top 3 schedules in this generation:
  Rank 1: Fitness = 575
  Rank 2: Fitness = 565
  Rank 3: Fitness = 550
Generation 6/100: Best Fitness = 575, Time: 0.81s
Top 3 schedules in this generation:
  Rank 1: Fitness = 575
  Rank 2: Fi